La siguiente celda se encargara de importar todos los paquetes necesarios para el desarrollo del proyecto. Además, se importa el archivo de configuración de la base de datos.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import ltrim,rtrim,trim,col
from pyspark.sql.functions import *
from pyspark import SparkContext
spark = SparkSession.builder.config("spark.jars", "postgresql-42.5.1.jar") \
	.master("local").appName("PySpark_Postgres_test").getOrCreate()

En la siguiente celda se encarga de utilizar spark para cargar en dos diferentes dataframes los datos del INEC y el OIJ para despues limpiarlos y unirlos en un solo dataframe.

In [ ]:
#CREATE DATAFRAME FROM CSV FILE
oij_df = spark.read.csv( path="OIJ.csv", sep=",", header=True,quote='"',inferSchema=True,)
inec_df = spark.read.csv( path="INEC.csv", sep=";", header=True,quote='"',inferSchema=True,)

En la siguiente celda se encarga de realizar la limpieza de los datos del INEC y el OIJ. Primero se creo una función para remover todos los espacios en blanco al inicio y al final de cada sub celda del data frame utilizando la funcion trim para ello. Ademas se creo una funcio la cual se encarga de pasar todos los caracteres a letras minusculas para poder realizar la busqueda de los datos en el data frame. 

In [ ]:
#Function to remove spaces from the beginning and end of the string
def remove_spaces(df):
    for col in df.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito' or col == 'Provincia, cantón y distrito':
            df = df.withColumn(col, trim(col))
    return df

#Function to parser the string to lowercase
def to_lower_case(df):
    for col in df.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito' or col == 'Provincia, cantón y distrito':
            df = df.withColumn(col, lower(col))
    return df


oij_df = remove_spaces(oij_df)
inec_df = remove_spaces(inec_df)

oij_df = to_lower_case(oij_df)
inec_df = to_lower_case(inec_df)

oij_df.show(5)
inec_df.show(5)

En la siguente celda se encuentra una funcion la cual se encarga de revisiar en los dos data frames aquellos datos donde no coinciden los datos de la provincia, canton y distrito. 

In [ ]:
#Function find list the not match values in the dataframes
def find_non_matches(df1,df2):
    non_matches = []
    for col in df1.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito':
            for row in df1.select(col).distinct().collect():
                if not df2.filter(df2['Provincia, cantón y distrito'] == row[col]).collect():
                    non_matches.append(row[col])
    return non_matches

non_matches_oij = find_non_matches(oij_df,inec_df)



oij_df = remove_spaces(oij_df)
inec_df = remove_spaces(inec_df)

oij_df = to_lower_case(oij_df)
inec_df = to_lower_case(inec_df)

nonMatches = find_non_matches(oij_df,inec_df)
print(nonMatches)
#oij_df.show(5)
#inec_df.show(5)


En la siguiente celda se encarga de reemplazar todos los caracteres especiales del español por caracteres ascii para poder realizar la busqueda de los datos en el data frame. Y que su taza de concidencia sea mayor.Para este trabajo se utilizo la funcionalidad regex_replace de spark. Y asi cada vez que se encuentre un caracter especial se reemplazara por un caracter ascii.

In [ ]:
#Function to replace the accents in column Provincia, cantón y distrito in inec_df
def replace_accents(df):
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'á', 'a'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'é', 'e'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'í', 'i'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'ó', 'o'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'ú', 'u'))
    df = df.withColumn('Provincia, cantón y distrito', regexp_replace('Provincia, cantón y distrito', 'ñ', 'n'))
    return df

inec_df = replace_accents(inec_df)
nonMatches = find_non_matches(oij_df,inec_df)
print(nonMatches)


En la siguiente celda se encarga de dividir la columna del dataframe del inec en tres columnas diferentes, una para la provincia, otra para el canton y otra para el distrito. Para esto se creo una funcion que basado en la cantidad espacios que se encuentren en la columna se dividira en tres columnas diferentes. Esto para que la comparacion de los datos sea mas exacta.

In [ ]:
def generate_new_columns(df):
    columns = ["Provincia1", "Canton1", "Distrito1","Tasa neta de participación", "Porcentaje de población económicamente inactiva", "Relación de dependencia económica"]
    new_df = spark.createDataFrame(data =[("","","","","","")], schema = columns)
    Provincia =''
    Canton = ''
    Distrito = ''
    counter = 0
    counter2 = 0
    for row in df.collect():
        if row['Provincia, cantón y distrito'] == None:
            counter += 1
            counter2 += 1
            if counter == 4: 
                counter = 2  

            if counter2 == 2:
                counter = 1
            continue
        if counter == 1:
            Provincia = row[0]
        if counter == 2:
            Canton = row[0]
        if counter == 3:
            Distrito = row[0]
            NewRow = (Provincia, Canton, Distrito, "", "", "")
            new_df = new_df.union(spark.createDataFrame(data =[NewRow], schema = columns))
        counter2 = 0
    
    return new_df

new_df = generate_new_columns(inec_df)

Esta funcion todavia no se si se deberia utilizar 

En la siguiente celda se encarga de la conexion a la base de datos y la creacion de las tablas necesarias para el almacenamiento de los datos ya limpios y procesados. Se crean dos tablas una para el inec y otra para el oij.

In [ ]:
#Create sql table from dataframe
url = "jdbc:postgresql://localhost:5432/etl"
mode = "overwrite"
properties = {"user": "postgres", " password": "Legolas00", "driver": "org.postgresql.Driver"}

new_df.write.jdbc(url=url, table="INEC", mode=mode, properties=properties)
oij_df.write.jdbc(url=url, table="OIJ", mode=mode, properties=properties)